In [1]:
from manim import *
import textwrap

class Manim3dCoordinatesAndCamera(ThreeDScene):
    def construct(self):
        # ==========================================
        # 1. CONFIGURATION
        # ==========================================
        self.frame_shift = LEFT * 4.0
        self.camera_zoom = 0.6
        self.ui_bg_pos = LEFT * 6.5 
        
        # ==========================================
        # 2. UI LAYER
        # ==========================================
        self.setup_fixed_ui()
        
        # ==========================================
        # 3. INITIALIZE 3D CAMERA
        # ==========================================
        # Start at "Front View"
        # Phi=90 (Horizon), Theta=-90 (Facing Front)
        self.set_camera_orientation(
            phi=90 * DEGREES, 
            theta=-90 * DEGREES,
            zoom=self.camera_zoom,
            frame_center=self.frame_shift
        )
        
        # ==========================================
        # 4. ANIMATION SEQUENCE
        # ==========================================
        self.run_setup_scene()
        self.run_theta_demo() # Spin (Yellow path)
        self.run_phi_demo()   # Tilt (Red path)
        self.run_zoom_demo()  # Zoom
        
        self.update_code("Tutorial Complete.")
        self.wait(2)

    def setup_fixed_ui(self):
        # 1. Black Background
        self.ui_bg = Rectangle(
            width=7.1, height=10, 
            color=BLACK, fill_color=BLACK, fill_opacity=1, stroke_width=0
        )
        self.ui_bg.move_to(self.ui_bg_pos)
        self.ui_bg.set_z_index(10)
        
        # 2. Gray Border
        self.code_border = Rectangle(
            width=6.5, height=7.0, color=GRAY, stroke_width=2
        )
        self.code_border.move_to(self.ui_bg_pos)
        self.code_border.set_z_index(11)
        
        # 3. Titles
        self.title = Text("Manim Tutorial: 3D Axes and Camera", font_size=32)
        self.title.to_edge(UP, buff=0.1).shift(2*LEFT)
        self.title.set_z_index(12)
        
        self.t_code = Text("Source Code", font_size=20, color=GRAY)
        self.t_code.next_to(self.code_border, UP, aligned_edge=LEFT)
        self.t_code.set_z_index(12)
        
        # 4. Attach to Screen
        self.add_fixed_in_frame_mobjects(
            self.ui_bg, self.code_border, self.title, self.t_code
        )

    def update_code(self, code_text):
        if hasattr(self, 'code_group'):
            self.remove_fixed_in_frame_mobjects(self.code_group)
            self.remove(self.code_group)
            
        lines = textwrap.dedent(code_text).strip().split('\n')
        self.code_group = VGroup()
        
        start_point = self.code_border.get_top() + DOWN * 0.5 + LEFT * 2.8
        
        for i, line in enumerate(lines):
            c = WHITE
            if "phi" in line or "theta" in line: c = ORANGE
            if "zoom" in line: c = YELLOW
            if "#" in line: c = GREY
            
            t = Text(line, font="Monospace", font_size=18, color=c)
            t.move_to(start_point + DOWN * (i * 0.4), aligned_edge=LEFT)
            self.code_group.add(t)
            
        self.code_group.set_z_index(13)
        self.add_fixed_in_frame_mobjects(self.code_group)

    # ---------------------------------------------------------
    # SCENE CHAPTERS
    # ---------------------------------------------------------
    def run_setup_scene(self):
        self.update_code("""
        # 1. Setup Scene
        # Front View (Side)
        # Phi=90, Theta=-90
        
        axes = ThreeDAxes()
        sphere = Sphere(r=1.5)
        self.add(labels)
        """)
        
        self.axes = ThreeDAxes()
        self.axes.move_to(LEFT * 4)
        
        self.sphere = Sphere(
            center=self.axes.get_center(), radius=1.5,
            checkerboard_colors=[BLUE_D, BLUE_E], stroke_width=1
        )
        
        labels = self.axes.get_axis_labels(
            Text("X", color=RED), 
            Text("Y", color=GREEN).scale(0.45), 
            Text("Z", color=BLUE).scale(0.45)
        )
        
        self.play(Create(self.axes), Create(self.sphere))
        self.add(labels)
        self.wait(1)

    def run_theta_demo(self):
        self.update_code("""
        # 2. Theta (Spin)
        # Phi stays at 90
        # Theta rotates to -45
        
        # (Camera orbits around Z)
        self.move_camera(
           theta = -45 * DEGREES,
           run_time=3
        )
        """)
        
        # --- VISUAL GUIDE CURVE ---
        # Arc on the floor (XY plane) from -90 to -45
        path_arc = Arc(
            radius=4,
            start_angle=-90*DEGREES,
            angle=45*DEGREES,
            color=YELLOW,
            stroke_width=6
        )
        path_arc.move_arc_center_to(self.axes.get_center())
        path_arc.add_tip()
        
        lbl = Text("Theta Move", color=YELLOW, font_size=24)
        lbl.next_to(path_arc, LEFT)
        # Since text is 2D, we must rotate it to lie flat on the "floor"
        # so it looks correct before the camera moves
        lbl.rotate(90*DEGREES, axis=RIGHT) 
        lbl.rotate(90*DEGREES, axis=OUT)
        
        self.play(Create(path_arc), Write(lbl))
        self.wait(0.5)

        # --- MOVE CAMERA ---
        self.move_camera(
            theta=-45 * DEGREES,
            phi=90 * DEGREES,
            zoom=self.camera_zoom,
            frame_center=self.frame_shift,
            run_time=3
        )
        self.play(FadeOut(path_arc), FadeOut(lbl))
        self.wait(1)

    def run_phi_demo(self):
        self.update_code("""
        # 3. Phi (Tilt)
        # Theta stays at -45
        # Phi rotates 90 -> 45
        
        # (Camera lifts up)
        self.move_camera(
           phi = 45 * DEGREES,
           run_time=3                         
        )
        """)
        
        # --- VISUAL GUIDE CURVE ---
        # Vertical Arc.
        # 1. Create flat arc
        # 2. Rotate 90 deg about X to stand it up
        # 3. Rotate -45 deg about Z to match our current viewing angle
        
        path_arc = Arc(
            radius=4,
            start_angle=0,          # Start at horizon
            angle=45*DEGREES,       # Lift up 45 degrees
            color=RED,
            stroke_width=6
        )
        path_arc.move_arc_center_to(self.axes.get_center())
        
        # Rotate to stand vertical
        path_arc.rotate(90*DEGREES, axis=RIGHT)
        # Rotate to align with current camera angle (theta = -45)
        path_arc.rotate(-45*DEGREES, axis=Z_AXIS)
        path_arc.add_tip()
        
        lbl = Text("Phi Move", color=RED, font_size=24)
        # Position label near the top of the arc
        # We manually position it in 3D space roughly where the arc ends
        # Coordinates: Radius * cos/sin math, or just manual approximation
        # x = r * cos(-45), y = r * sin(-45), z = height
        lbl.move_to(self.axes.get_center() + 5*UP + 5*LEFT)
        
        # Billboard the text (face the camera)
        # Since we are at theta=-45, phi=90, we rotate text to face us
        lbl.rotate(90*DEGREES, axis=RIGHT)
        lbl.rotate(-45*DEGREES, axis=OUT)

        self.play(Create(path_arc), Write(lbl))
        self.wait(0.5)

        # --- MOVE CAMERA ---
        self.move_camera(
            phi=45 * DEGREES,
            theta=-45 * DEGREES,
            zoom=self.camera_zoom,
            frame_center=self.frame_shift,
            run_time=3
        )
        self.play(FadeOut(path_arc), FadeOut(lbl))
        self.wait(1)

    def run_zoom_demo(self):
        self.update_code("""
        # 4. Zoom (Dolly)
        # Angles stay same.
        # Zoom increases to 1.0
        
        self.move_camera(
           zoom = 1.0,
           run_time=3
        )
        """)
        
        self.move_camera(
            zoom=1.0,
            phi=45 * DEGREES,
            theta=-45 * DEGREES,
            frame_center=self.frame_shift,
            run_time=3
        )
        self.wait(1)
        
        # Reset
        self.move_camera(zoom=self.camera_zoom, frame_center=self.frame_shift, run_time=1)

%manim -qk -v warning Manim3dCoordinatesAndCamera

Manim Community v0.19.1